# Pratice 2: Quantum Support Vector Machines

MQIST 2025/26: Quantum Computing and Machine Learning

Alfredo Chavert Sancho

Pedro Herrero Maldonado

## Data loading and preprocessing

The `load_breast_cancer` function from `sklearn.datasets` is used to load the breast cancer dataset. This dataset contains features computed from breast cancer biopsy images, along with labels indicating whether the cancer is malignant or benign. So this is a binary classification problem. There are 30 features in total and 569 samples which are divided into:

- 357 benign samples labeled as 1
- 212 malignant samples labeled as 0

As measure to evaluate the performance of our quantum neural network, we will use **recall** of the positive class labeled with 0, which is the ratio of correctly predicted positive malignant cases to the total actual positive instances in the dataset (true positives and false negatives). 
$$
\text{Recall} = \frac{\text{True Malignant}}{\text{True Malignant} + \text{False Benign}}
$$

This is particularly important in medical diagnosis tasks, where minimizing false negatives (i.e., failing to identify actual positive cases) is crucial. Therefore, our goal in this work is to maximize this metric in our experiments.

In [1]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X = data.data # features
y = data.target # labels

print("Original shape:", X.shape)

Original shape: (569, 30)


For the prepocessing the data to be used in the quantum neural network, we will use  ``StandardScaler`` from ``sklearn.preprocessing`` to normalize it  since all the data is numeric.

We will also use ``PCA`` from ``sklearn.decomposition`` to reduce the dimensionality of the data to reduce the number of qubits needed in the quantum neural network. This transformation helps in capturing the most significant variance in the data while reducing its complexity.

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Standardize (important for PCA)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA (ex: reduce to 2 components for visualization or QML input)
pca = PCA(n_components = 4)
X_pca = pca.fit_transform(X_scaled)

print("Shape after PCA:", X_pca.shape) # (150, 2)
print("Explained variance:", pca.explained_variance_ratio_)
print("Total variance explained:", pca.explained_variance_ratio_.sum())

Shape after PCA: (569, 4)
Explained variance: [0.53097689 0.1728349  0.07114442 0.06411259]
Total variance explained: 0.8390687984671886


Finally, we will split the dataset into training and testing sets using ``train_test_split`` from ``sklearn.model_selection``.

In [4]:
from sklearn.model_selection import train_test_split
from qiskit_algorithms.utils import algorithm_globals

# Set the seed so results are reproducible
algorithm_globals.random_seed = 42

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=algorithm_globals.random_seed,)

## Classical SVM

### Linear kernel

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

svc_linear = SVC(kernel = 'linear', gamma='auto')
svc_linear.fit(X_train, y_train)

y_svc_linear = svc_linear.predict(X_test)
report_svc_linear = classification_report(y_test, y_svc_linear)
print(report_svc_linear)

              precision    recall  f1-score   support

           0       0.95      0.93      0.94        43
           1       0.96      0.97      0.97        71

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



### Polynomial kernel

In [6]:
svc_poly = SVC(kernel = 'poly', gamma='auto')
svc_poly .fit(X_train, y_train)

y_svc_poly = svc_poly.predict(X_test)
report_svc_poly = classification_report(y_test, y_svc_poly)
print(report_svc_poly)

              precision    recall  f1-score   support

           0       1.00      0.23      0.38        43
           1       0.68      1.00      0.81        71

    accuracy                           0.71       114
   macro avg       0.84      0.62      0.59       114
weighted avg       0.80      0.71      0.65       114



### RBF kernel

In [7]:
svc_rbf = SVC(kernel = 'rbf', gamma='auto')
svc_rbf .fit(X_train, y_train)

y_svc_rbf = svc_rbf.predict(X_test)
report_svc_rbf = classification_report(y_test, y_svc_rbf)
print(report_svc_rbf)

              precision    recall  f1-score   support

           0       1.00      0.23      0.38        43
           1       0.68      1.00      0.81        71

    accuracy                           0.71       114
   macro avg       0.84      0.62      0.59       114
weighted avg       0.80      0.71      0.65       114



## Quantum SVM

### Z Feature Map

In [8]:
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC
from qiskit.circuit.library import ZFeatureMap

num_features = X_pca.shape[1]

kernel = FidelityQuantumKernel(feature_map = ZFeatureMap(feature_dimension = num_features, reps = 2))
qsvc_z = QSVC(quantum_kernel = kernel)
qsvc_z.fit(X_train, y_train)
y_pred_qsvc_z = qsvc_z.predict(X_test)

report_qsvc_z = classification_report(y_test, y_pred_qsvc_z)
print(report_qsvc_z)

              precision    recall  f1-score   support

           0       0.89      0.91      0.90        43
           1       0.94      0.93      0.94        71

    accuracy                           0.92       114
   macro avg       0.91      0.92      0.92       114
weighted avg       0.92      0.92      0.92       114



### ZZ Feature Map

In [9]:
from qiskit.circuit.library import ZZFeatureMap

kernel = FidelityQuantumKernel(feature_map = ZZFeatureMap(feature_dimension = num_features, reps = 2))
qsvc_zz = QSVC(quantum_kernel = kernel)
qsvc_zz.fit(X_train, y_train)
y_pred_qsvc_zz = qsvc_zz.predict(X_test)

report_qsvc_zz = classification_report(y_test, y_pred_qsvc_zz)
print(report_qsvc_zz)

              precision    recall  f1-score   support

           0       0.80      0.77      0.79        43
           1       0.86      0.89      0.88        71

    accuracy                           0.84       114
   macro avg       0.83      0.83      0.83       114
weighted avg       0.84      0.84      0.84       114



### Pauli Feature Map

In [ ]:
from qiskit.circuit.library import PauliFeatureMap

kernel = FidelityQuantumKernel(feature_map = PauliFeatureMap(feature_dimension = num_features, reps = 2, paulis = ['Z', 'XX', 'ZXZ']))
qsvc_pauli = QSVC(quantum_kernel = kernel)
qsvc_pauli.fit(X_train, y_train)
y_pred_qsvc_pauli = qsvc_pauli.predict(X_test)

report_qsvc_pauli = classification_report(y_test, y_pred_qsvc_pauli)
print(report_qsvc_pauli)